In [1]:
import cstl_ntwkx
import matplotlib.pyplot as plt
import gymnasium as gym
import networkx as nx
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from PIL import Image

Import dependencies

Set constellation parameters

In [8]:
from typing import List, Tuple


sats = 40
planes = 4
ipc = 1
altitude = 1200
inclination = 86.0
type = cstl_ntwkx.ConstellationType.Star

cstl_ntwkx.plot.test()

ground_stations: List[Tuple[str, float, float, float]] = [
    # (Name, lat, lon, alt)
    ("Saarbrücken", 49.233, 7.0, 0.23),
    ("Washington DC", 38.889805, -77.009056, 0.125),
]

AttributeError: module 'cstl_ntwkx' has no attribute 'plot'

Define source and target node

In [3]:
source_node = 0
target_node = 2

Validate params and calculate implied parameters

In [4]:
assert sats % planes == 0
assert source_node >= 0 and source_node < sats
assert target_node >= 0 and target_node < sats

sats_per_plane = int(sats / planes)

Create the constellation

In [5]:
cstl = cstl_ntwkx.create_constellation(
    sats, planes, ipc, altitude, inclination, 10.0, type
)

# add ground stations
for (name, lat, lon, alt) in ground_stations:
    cstl.add_groundstation(name, lat, lon, alt)

cstl.propagate(25_520_000)

Extract the Graph

In [6]:
G: nx.Graph = cstl_ntwkx.extract_graph(cstl)

node_positions = cstl_ntwkx.project_3d_positions(cstl)


Project nodes on map

In [7]:
sat_image = Image.open("images/satellit_blue.gif")
gs_image = Image.open("images/gs.png")
bg_image = Image.open("images/background_earth.png")
img_width = 800
img_height = 400

icon_width = 0.04
icon_height = 0.04

In [8]:
# Create figure
fig = go.Figure()

# Configure axes
fig.update_xaxes(visible=False, range=[0, img_width])

fig.update_yaxes(visible=False, range=[0, img_height])

# Add nodes to map (icons)
projected_positions = dict()

for sat in node_positions:
    (typ, (lat, lon, alt)) = node_positions[sat]
    x = (lon + 180.0) / 360.0
    x_canvas = x * img_width
    y = (lat + 90.0) / 180.0
    y_canvas = y * img_height
    if typ == "S":
        img = sat_image
    elif typ == "G":
        img = gs_image
    else:
        raise ValueError(f"Node type {typ} is unknown!")

    fig.add_layout_image(
        dict(
            source=img,
            xanchor="center",
            yanchor="middle",
            x=x,
            y=y,
            sizex=icon_width,
            sizey=icon_height,
        )
    )

    fig.add_annotation(
        x=x_canvas,
        y=y_canvas,
        text=sat,
        showarrow=False,
        yshift=-10,
        font=dict(family="sans serif", size=18, color="white"),
    )

    # print("Add annotation for", sat, "at", x_canvas, y_canvas)

    # store projected position
    projected_positions[sat] = (x_canvas, y_canvas)

# add connections to map
for src, dst in G.edges:
    (typ_0, (lat_0, lon_0, alt_0)) = node_positions[src]
    (typ_1, (lat_1, lon_1, alt_1)) = node_positions[dst]
    src_x_canvas, src_y_canvas = projected_positions[src]
    dst_x_canvas, dst_y_canvas = projected_positions[dst]

    # print(f"Connect {typ_0}-{src}({x_src}/{y_src}) and {typ_1}-{dst}({x_dst}/{y_dst})")

    left_x = min(src_x_canvas, dst_x_canvas)
    right_x = max(src_x_canvas, dst_x_canvas)
    if src_x_canvas > dst_x_canvas:
        right_y = src_y_canvas
        left_y = dst_y_canvas
    else:
        right_y = dst_y_canvas
        left_y = src_y_canvas

    simple_distance = abs(src_x_canvas - dst_x_canvas)
    modulo_distance = left_x + img_width - right_x

    if modulo_distance < simple_distance:
        cut_point = abs(src_y_canvas - dst_y_canvas)
        cut_point = min(left_y, right_y) + (cut_point / 2)
        # print(f"{src}({src_y_canvas}) -> {dst}({dst_y_canvas}) (CP: {cut_point})")
        fig.add_trace(
            go.Scatter(
                x=[right_x, img_width],
                y=[right_y, cut_point],
                line_color="red",
                mode="lines",
            )
        )
        fig.add_trace(
            go.Scatter(
                x=[0, left_x], y=[cut_point, left_y], line_color="red", mode="lines"
            )
        )
    else:
        fig.add_trace(
            go.Scatter(
                x=[src_x_canvas, dst_x_canvas],
                y=[src_y_canvas, dst_y_canvas],
                line_color="red",
            )
        )

    # print(f"({src}-{dst}) = Simple: {simple_distance}; Modulo: {modulo_distance}")


fig.update_layout(
    margin={"l": 0, "t": 0, "b": 0, "r": 0},
    width=img_width,
    height=img_height,
    hovermode=False,
    dragmode=False,
    showlegend=False,
)

# Add image
fig.add_layout_image(
    dict(
        source=bg_image,
        xref="x",
        yref="y",
        x=0,
        sizex=img_width,
        y=img_height,
        sizey=img_height,
        layer="below",
        sizing="stretch",
    )
)
fig.show(config={"displayModeBar": False})

In [9]:
# node_positions = [
#     (plane * sats_per_plane + sat_in_plane, (plane, sat_in_plane))
#     for sat_in_plane in range(sats_per_plane)
#     for plane in range(planes)
# ]

# cstl = cstl_ntwkx.propagate(cstl, 1000)


# print(G)
# print(node_positions)

# # Create the 3D figure
# fig = plt.figure()
# ax = fig.add_subplot(111, projection="3d")

# # plot satellites
# for pos in node_positions:
#     (xs, ys, zs) = node_positions[pos]
#     ax.scatter(xs, ys, zs, s=100, ec="w")

# for src, dst in G.edges:
#     (s_x, s_y, s_z) = node_positions[src]
#     (d_x, d_y, d_z) = node_positions[dst]
#     ax.plot([s_x, d_x], [s_y, d_y], [s_z, d_z], color="tab:gray")

# shortest_path = nx.dijkstra_path(G, source_node, target_node, weight='weight')
# path_edges = list(zip(shortest_path, shortest_path[1:]))
# print(path_edges)
# for (s, d) in path_edges:
#     G.add_edge(u_of_edge=s, v_of_edge=d, color="tab:blue")
# plt.title("Constellation")
# ax.set_xlabel("X-axis (km)")
# ax.set_ylabel("Y-axis (km)")
# ax.set_zlabel("Z-axis (km)")
# ax.xaxis.set_tick_params(labelsize=7)
# ax.yaxis.set_tick_params(labelsize=7)
# ax.zaxis.set_tick_params(labelsize=7)
# ax.set_aspect("equal", adjustable="box")
# plt.show()

# CHANGE THESE VALUES IF YOU ARE NOT ORBITING EARTH !
# mu = 398600.4418
# r = 6781
# D = 24 * 0.997269

# fig = plt.figure()
# ax = plt.axes(projection="3d", computed_zorder=False)

# # plot satellites
# for pos in node_positions:
#     (xs, ys, zs) = node_positions[pos]
#     ax.scatter(xs, ys, zs)

# for src, dst in graph.edges:
#     (s_x, s_y, s_z) = node_positions[src]
#     (d_x, d_y, d_z) = node_positions[dst]
#     ax.plot3D([s_x, d_x], [s_y, d_y], [s_z, d_z])

# # then we plot the earth
# u, v = np.mgrid[0 : 2 * np.pi : 20j, 0 : np.pi : 10j]
# ax.plot_surface(
#     r * np.cos(u) * np.sin(v),
#     r * np.sin(u) * np.sin(v),
#     r * np.cos(v),
#     color="b",
#     alpha=1,
#     lw=0.5,
#     zorder=0,
# )
# plt.title("Constellation")
# ax.set_xlabel("X-axis (km)")
# ax.set_ylabel("Y-axis (km)")
# ax.set_zlabel("Z-axis (km)")
# ax.xaxis.set_tick_params(labelsize=7)
# ax.yaxis.set_tick_params(labelsize=7)
# ax.zaxis.set_tick_params(labelsize=7)
# ax.set_aspect("equal", adjustable="box")
# fig.subplots_adjust(right=0.8)
# ax.legend(loc="center left", bbox_to_anchor=(1.07, 0.5), fontsize=7)
# plt.show()
